
# PyTorch Quickstart

Have a pytorch model? This notebook will show you how to get it set up in Lens. If this is your first time using Lens, we suggest starting with our [quickstart notebook](https://credoai-lens.readthedocs.io/en/stable/notebooks/quickstart.html) which takes you through the basics with an sklearn model.

**Setup**

Lens installation instruction can be found on [readthedocs](https://credoai-lens.readthedocs.io/en/stable/pages/setup.html)

**Find the code**

### Get your pytorch model and dataloaders ready.
 In this tutorial we will emulate the modeling phase by running a quick script. This script following the [MNIST quickstart](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html) from the pytorch documentation. You can see the full script [here](https://raw.githubusercontent.com/credo-ai/credoai_lens/main/credoai/demo_assets/_fetch_pytorch_mnist_model.py).


Because one of the primary use cases for Lens is to easily add Fairness and bias assessment to your pipeline, we are going to make up a sensitive features array
for the purpose of demonstration.

In [2]:
# model and X_test, y_test, etc. are defined by this script
from credoai.demo_assets import fetch_mnist_model

# model and data are defined by this script
model, train_dataloader, test_dataloader = fetch_mnist_model()

# making up some sensitive features
import numpy as np
import pandas as pd
sensitive_categories = ['black', 'white']
sensitive_features = pd.Series(np.random.choice(sensitive_categories, size=len(test_dataloader.dataset)),
                                name='color')

Using mps device
Get MNIST data
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Define the model
Train the model
Epoch 1
-------------------------------
loss: 2.311020  [   64/60000]
loss: 2.299083  [ 6464/60000]
loss: 2.270861  [12864/60000]
loss: 2.265982  [19264/60000]
loss: 2.252087  [25664/60000]
loss: 2.234039  [32064/60000]
loss: 2.237553  [38464/60000]
loss: 2.212780  [44864/60000]
loss: 2.209331  [51264/60000]
loss: 2.178882  [57664/60000]
Epoch 2
-------------------------------
loss: 2.177557  [   64/60000]
loss: 2.167854  [ 6464/60000]
loss: 2.104097  [12864/60000]
loss: 2.129525  [19264/60000]
loss: 2.082517  [25664/60000]
loss: 2.030308  [32064/60000]
loss: 2.066048  [38464/60000]
loss: 1.988355  [44864/60000]
loss: 1.995682  [51264/60000]
loss: 1.940706  [57664/60000]
Epoch 3
-------------------------------
loss: 1.942122  [   64/60000]
loss: 1.919970  [ 6464/60000]
loss: 1.792076  [12864/60000]
loss: 1.855927  [19264/60000]
l

### Lens

Below is a basic example where our goal is to evaluate the above model.

Briefly, the code is doing four things:

* Wrapping ML artifacts (like models and data) in Lens objects
* Initializing an instance of Lens. Lens is the main object that performs evaluations. Under the hood, it creates a `pipeline` of evaluations that are run.
* Add evaluators to Lens.
* Run Lens

In [3]:
# Import Lens and necessary artifacts
from credoai.lens import Lens
from credoai.artifacts import ClassificationModel, PytorchData

deepchecks - WARNING - You are using deepchecks version 0.11.1, however a newer version is available.Deepchecks is frequently updated with major improvements. You should consider upgrading via the "python -m pip install --upgrade deepchecks" command.


In lens, the classes that evaluate models and/or datasets are called `evaluators`. In this example we are interested in evaluating the model's fairness. For this we can use the `ModelFairness` evaluator. We'll
also evaluate the model's performance.

In [ ]:
from credoai.evaluators import ModelFairness, Performance

## Lens in 5 minutes

Below is a basic example where our goal is to evaluate the above model. We'll break down this code [below](#Breaking-Down-The-Steps).

Briefly, the code is doing four things:

* Wrapping ML artifacts (like models and data) in Lens objects
* Initializing an instance of Lens. Lens is the main object that performs evaluations. Under the hood, it creates a `pipeline` of evaluations that are run.
* Add evaluators to Lens.
* Run Lens

In [31]:
model = 1

In [32]:
model.__class__.__bases__[0].__module__.startswith('torch')

False

In [ ]:
credo_data = PytorchData(
    name = 'mnist-data',
    dataloader=test_dataloader,
    sensitive_features=sensitive_features
)

credo_model = ClassificationModel(
    name='mnist-classifier',
    model_like=model,
    tags=None
)

# Initialization of the Lens object
lens = Lens(model=credo_model, assessment_data=credo_data)

# initialize the evaluator and add it to Lens
metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics))
lens.add(Performance(metrics=metrics))

# run Lens
lens.run()

2022-10-31 14:43:01,052 - lens - INFO - Evaluator ModelFairness added to pipeline. Sensitive feature: SEX
2022-10-31 14:43:01,243 - lens - INFO - fairness metric, equal_opportunity, unused by PerformanceModule
2022-10-31 14:43:01,250 - lens - INFO - Evaluator Performance added to pipeline. 
2022-10-31 14:43:01,250 - lens - INFO - Running evaluation for step: PipelineStep(evaluator=<credoai.evaluators.fairness.ModelFairness object at 0x2a5a20fd0>, metadata={'evaluator': 'ModelFairness', 'sensitive_feature': 'SEX', 'dataset_type': 'assessment_data'})
2022-10-31 14:43:01,259 - lens - INFO - Running evaluation for step: PipelineStep(evaluator=<credoai.evaluators.performance.Performance object at 0x2a5a3dac0>, metadata={'evaluator': 'Performance'})
